In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pylab as plt
import sys, gc, warnings, random, math, time, datetime 
from tqdm import tqdm
START_DATE = datetime.datetime.strptime('2017-11-30', '%Y-%m-%d')

from sklearn import preprocessing
from sklearn import metrics
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error

import xgboost as xgb
import lightgbm as lgb

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

from utils import *

import os
print(os.listdir("../input/mini-model-data/"))

['test_df_h.pkl', 'test_df_c.pkl', 'train_df_w.pkl', 'test_df_w.pkl', 'train_df_s.pkl', 'test_df_s.pkl', 'test_df_r.pkl', 'train_df_h.pkl', 'test_identity.pkl', 'train_df_r.pkl', 'train_identity.pkl', 'train_df_c.pkl']


In [2]:
train = pd.read_pickle('../input/mini-model-features/train_features_r.pkl')
test= pd.read_pickle('../input/mini-model-features/test_features_r.pkl')

train_y = train['isFraud'].copy()

print(train.shape)
print(train_y.shape)
print(test.shape)

(37699, 382)
(37699,)
(35647, 382)


In [3]:
rm_cols = [
    'isFraud', 'TransactionDT', 'DT', 'addr2', 'C6', 'TransactionAmt', 'id_18'
    'id_21', 'id_22','id_30', 'id_31', 'id_34',
   'V28', 'V65','V88',
    'DeviceInfo', 
]
features_columns = list(train)
for col in rm_cols:
    if col in features_columns:
        features_columns.remove(col)

In [4]:
########################### Model params
lgb_params = {
                    'objective':'binary',
                    'boosting_type':'gbdt',
                    'metric':'auc',
                    'n_jobs':-1,
                    'learning_rate':0.01,
                    'num_leaves': 2**8,
                    'max_depth':-1,
                    'tree_learner':'serial',
                    'colsample_bytree': 0.7,
                    'subsample_freq':1,
                    'subsample':0.7,
                    'n_estimators':1000,
                    'max_bin':255,
                    'verbose':-1,
                    'seed': 42,
                    'early_stopping_rounds':100, 
                } 

In [5]:
test_predictions, auc_score = make_predictions(train, test, train_y, features_columns, lgb_params)

Fold: 0  -  17373 20326
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[380]	training's auc: 1	valid_1's auc: 0.935483
Fold: 1  -  33498 4201
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[291]	training's auc: 1	valid_1's auc: 0.975774
Fold: 2  -  34254 3445
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[216]	training's auc: 1	valid_1's auc: 0.971982
Fold: 3  -  34347 3352
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[323]	training's auc: 1	valid_1's auc: 0.964259
Fold: 4  -  34505 3194
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[325]	training's auc: 1	valid_1's auc: 0.960455
Fold: 5  -  34518 3181
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[275]	training's auc: 1	valid_1's

In [6]:
test_predictions.to_pickle('../input/mini-model-predictions/predictions_r.pkl')